In [11]:
import os 

import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

data_path = Path('../input/split_2-1')
output_path = Path('../input/split_2-1_pqt')
os.mkdir(output_path)

chunksize = 100_000

In [12]:
chunks = pd.read_json(data_path / 'train_sessions.jsonl', lines=True, chunksize=chunksize)
os.mkdir(output_path / 'train_parquets')

for e, chunk in enumerate(tqdm(chunks, total=129)):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }

    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])

    # save DataFrame
    start = str(e*chunksize).zfill(9)
    end = str(e*chunksize+chunksize).zfill(9)
    pd.DataFrame(event_dict).to_parquet(output_path / 'train_parquets' / f"{start}_{end}.parquet")

 60%|██████    | 78/129 [03:44<02:26,  2.88s/it]


In [13]:
chunks = pd.read_json(data_path / 'test_sessions.jsonl', lines=True, chunksize=chunksize)
os.mkdir(output_path / 'test_parquets')

for e, chunk in enumerate(tqdm(chunks, total=17)):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }

    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])

    # save DataFrame
    start = str(e*chunksize).zfill(9)
    end = str(e*chunksize+chunksize).zfill(9)
    pd.DataFrame(event_dict).to_parquet(output_path / 'test_parquets' / f"{start}_{end}.parquet")

23it [00:29,  1.26s/it]                        


In [19]:
test_labels = pd.read_json(data_path / 'test_labels.jsonl', lines=True)
test_labels_new = {
        'session': [],
        'type': [],
        'ground_truth': [],
    }
for session, labels in tqdm(zip(test_labels.session, test_labels.labels)):
    for event in labels:
        test_labels_new['session'].append(session)
        test_labels_new['type'].append(event)
        if type(labels[event]) == int:
            test_labels_new['ground_truth'].append([labels[event]])
        else:
            test_labels_new['ground_truth'].append(labels[event])
pd.DataFrame(test_labels_new).to_parquet(output_path / 'test_labels.parquet')

,session,labels
0,8643220,{'clicks': 1189975}
1,8643221,{'clicks': 111691}
2,8643222,{'clicks': 756920}
3,8643223,{'clicks': 1742315}
4,8643224,"{'clicks': 778561, 'orders': [1502227, 154901,..."
...,...,...
2224029,11098472,{'clicks': 504876}
2224030,11098478,{'clicks': 261405}
2224031,11098483,{'clicks': 46115}
2224032,11098494,{'clicks': 1733630}
